### 题目 

Implement a job scheduler which takes in a function `f` and an integer `n`, and calls `f` after `n` milliseconds.

### 解答

In [8]:
from time import sleep
def scheduler(f, n):
    sleep(0.001*n)
    f()

In [17]:
def test_f1():
    print('test function 1')

def test_f2():
    print('test function 2')
    
def test_f3():
    print('test function 3')
scheduler(test_f1, 2000)
scheduler(test_f2, 4000)
scheduler(test_f3, 1000)

test function 1
test function 2
test function 3


上面的运行结果与我们期望的并不一样，只会按顺序执行。需要加入线程来解决这个问题

In [23]:
from time import sleep, time
import threading

class scheduler_fix:
    def __init__(self):
        self.fns = []
        t = threading.Thread(target=self.poll)
        t.start()
        
    def poll(self):
        while True:
            now = time() * 1000 
            for fn, due in self.fns:
                if now >= due:
                    fn()
            self.fns = [(fn, due) for fn, due in self.fns if due > now]
            sleep(0.01)
    def dely(self, f, n):
        self.fns.append((f, time() * 1000 + n ))

In [25]:
schedu_fix = scheduler_fix()
schedu_fix.dely(test_f1, 2000)
schedu_fix.dely(test_f2, 4000)
schedu_fix.dely(test_f3, 1000)

test function 3
test function 1
test function 2


但是每秒循环执行100次poll并不能很好的完成任务。通过加入锁，让程序变得更健壮。

In [15]:
from time import sleep, time
import threading

class scheduler_pro:
    def __init__(self):
        self.fns = []
        
        # 引入锁以保证线程安全，注意Lock与RLock的区别
        self.lock = threading.RLock()
        
        # 使用condition来挂起以及唤醒线程
        self.condition = threading.Condition(self.lock)
        
        t = threading.Thread(target=self.poll)
        t.start()
        
    def poll(self):
        
        while True:
            now = time() * 1000
            
            # with语句可以自动上锁、释放锁
            # 确保当前线程使用fns时其他线程不会修改fns
            with self.lock:
                
                to_run = [fn for fn, due in self.fns if due <= now ]
                self.fns = [(fn, due) for (fn, due) in self.fns if due > now]
                
            # 将任务分类好以后，执行需要运行的任务
            for fn in to_run:
                fn()
                
            with self.lock:
                
                if not self.fns:
                    # 当 fns 为空的时候 挂起线程，直到delay()的 notify 唤醒线程
                    self.condition.wait()
                else:
                    # 挂起线程直到最近一个任务的执行时间
                    ms_remaining = min(due for fn, due in self.fns) - time() * 1000
                    
                    if ms_remaining > 0:
                        self.condition.wait(ms_remaining/1000)
                
    def delay(self, f, n):
        with self.lock:
            self.fns.append((f, time() * 1000 + n ))
            self.condition.notify_all()

In [19]:
schedul_pro = scheduler_pro()
schedul_pro.delay(lambda:print('run after 3 seconds'), 3600)
schedul_pro.delay(lambda:print('run after 8 seconds'), 8800)
schedul_pro.delay(lambda:print('run after 5 seconds'), 5500)
schedul_pro.delay(lambda:print('run after 1 seconds'), 1300)

run after 1 seconds
run after 3 seconds
run after 5 seconds
run after 8 seconds
